The Goal here is to merge the episodes that be sure there is no negative LOE or TNE exists. The result of this process stored in data frame and CSV "full_episode"

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import seaborn as sns
import warnings

In [2]:
#load the data
episodes = pd.read_csv('/home/kabank/workbench/.conda/analysis/Resources/epiosdes2.csv')

In [3]:
episodes.rename(columns={'patient_age': 'age'}, inplace=True)

In [4]:
episodes.columns

Index(['pasient', 'episode_id', 'age', 'remaining_time_countdown',
       'var_no_dates_permonth', 'gender', 'episode_order', 'islast',
       'closingcode', 'aftercode', 'instanskode', 'episode_start_date',
       'episode_end_date', 'length_of_episode', 'tillnextepisode',
       'starttillnextstart', 'count_poliklinikk', 'count_familieavdeling',
       'count_dagavdeling', 'count_dognavdeling', 'count_osv',
       'count_outpatient', 'count_inpatient', 'count_lmssciadm',
       'count_inpatient_day', 'count_inpatient_daynight', 'count_terapi',
       'count_undersokelse', 'count_radge', 'count_behplanlegging',
       'count_ikkemott', 'count_others'],
      dtype='object')

In [5]:
episodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18646 entries, 0 to 18645
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   pasient                   18646 non-null  int64  
 1   episode_id                18646 non-null  int64  
 2   age                       18646 non-null  int64  
 3   remaining_time_countdown  18646 non-null  int64  
 4   var_no_dates_permonth     18646 non-null  float64
 5   gender                    18646 non-null  object 
 6   episode_order             18646 non-null  int64  
 7   islast                    18646 non-null  int64  
 8   closingcode               18646 non-null  int64  
 9   aftercode                 18646 non-null  int64  
 10  instanskode               18646 non-null  int64  
 11  episode_start_date        18646 non-null  object 
 12  episode_end_date          18646 non-null  object 
 13  length_of_episode         18646 non-null  int64  
 14  tillne

In [6]:
len(episodes['pasient'].unique()) #before 19682

15041

In [7]:
episodes[episodes['tillnextepisode']<=0]

,pasient,episode_id,age,remaining_time_countdown,var_no_dates_permonth,gender,episode_order,islast,closingcode,aftercode,...,count_inpatient,count_lmssciadm,count_inpatient_day,count_inpatient_daynight,count_terapi,count_undersokelse,count_radge,count_behplanlegging,count_ikkemott,count_others
28,20686,47,15,56,2.262983,F,2,0,1,2,...,0,0,0,0,64,4,8,3,1,3
46,4688,78,6,95,1.093532,F,1,0,1,5,...,0,0,0,0,32,25,14,7,0,1
57,5421,92,10,11,1.393510,F,1,0,1,1,...,0,0,0,0,83,32,3,10,3,4
130,10785,203,16,62,0.439321,F,6,0,4,2,...,2,0,0,2,14,2,2,2,3,0
151,8450,235,8,65,2.108185,M,1,0,1,1,...,0,0,0,0,46,53,37,17,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18576,10359,30820,16,122,0.987346,F,4,0,2,2,...,57,0,2,55,56,27,23,4,3,1
18591,8559,30841,10,137,1.530846,M,1,0,3,3,...,0,0,0,0,20,8,4,8,1,0
18615,469,30883,17,32,0.000000,F,4,0,1,5,...,13,0,0,13,1,8,4,0,0,0
18626,3118,30906,11,44,1.279922,M,1,0,1,1,...,0,0,0,0,19,25,9,7,1,2


In [8]:
episodes[episodes['length_of_episode']<=0]

,pasient,episode_id,age,remaining_time_countdown,var_no_dates_permonth,gender,episode_order,islast,closingcode,aftercode,...,count_inpatient,count_lmssciadm,count_inpatient_day,count_inpatient_daynight,count_terapi,count_undersokelse,count_radge,count_behplanlegging,count_ikkemott,count_others


In [9]:
# make a set for pasient having episodes with negative or zero value for tillnextepiosde
filter1 = episodes[episodes['tillnextepisode'] <= 0]
pasient_for_merge = filter1['pasient'].unique()

# Filter rows based on 'pasient_for_merge'
cases_of_merge = episodes[episodes['pasient'].isin(pasient_for_merge)]

# Filter rows not in 'pasient_for_merge'
norm_episodes = episodes[~episodes['pasient'].isin(pasient_for_merge)]

print("Cases of Merge:")
print(len(cases_of_merge))
print("\nNorm Episodes:")
print(len(norm_episodes))

Cases of Merge:
2507

Norm Episodes:
16139


In [10]:
# some exmaple for merge, pasient: 199 (2), 3878 (3), 469 (4)
# examples of Continuity and overlap: 6586, 19419, 126

# Selecting specific columns after filtering rows
# case test 1
cases_of_merge.loc[cases_of_merge['pasient'] == 1, 
                            ['episode_start_date', 'episode_end_date', 'episode_order', 'tillnextepisode']]

,episode_start_date,episode_end_date,episode_order,tillnextepisode


In [11]:
# case test 2
cases_of_merge.loc[cases_of_merge['pasient'] == 19419, 
                            ['episode_start_date', 'episode_end_date', 'episode_order', 'tillnextepisode']]

,episode_start_date,episode_end_date,episode_order,tillnextepisode


In [12]:
# case test 3
cases_of_merge.loc[cases_of_merge['pasient'] == 126, 
                            ['episode_start_date', 'episode_end_date', 'episode_order', 'tillnextepisode', 'length_of_episode']]

,episode_start_date,episode_end_date,episode_order,tillnextepisode,length_of_episode


In [13]:
merged_episodes = []
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for pasient in cases_of_merge['pasient'].unique():
        subset = episodes[episodes['pasient'] == pasient].sort_values(by=['episode_start_date', 'episode_end_date'], ascending=[True, False])
        i = 0
        while i < len(subset):
            outer = subset.iloc[i]
            merged = False
            j = i + 1
            while j < len(subset):
                inner = subset.iloc[j]
                if ((outer['episode_start_date'] <= inner['episode_start_date'] and 
                    outer['episode_end_date'] >= inner['episode_start_date'])):
                    # merge episodes
                    outer['remaining_time_countdown'] = min(outer['remaining_time_countdown'], inner['remaining_time_countdown'])
                    outer['var_no_dates_permonth'] = (outer['var_no_dates_permonth'] + inner['var_no_dates_permonth'])/2
                    outer['age'] = min(outer['age'], inner['age'])
                    outer['gender'] = max(outer['gender'], inner['gender'])
                    outer['closingcode'] = max(outer['closingcode'], inner['closingcode'])
                    outer['aftercode'] = max(outer['aftercode'], inner['aftercode'])
                    outer['islast'] = max(outer['islast'], inner['islast'])
                    outer['episode_order'] = max(outer['episode_order'], inner['episode_order'])
                    outer['instanskode'] = max(outer['instanskode'], inner['instanskode'])
                    outer['count_poliklinikk'] =  (outer['count_poliklinikk'] +  inner['count_poliklinikk'])
                    outer['count_familieavdeling'] = (outer['count_familieavdeling'] + inner['count_familieavdeling'])
                    outer['count_dagavdeling'] = (outer['count_dagavdeling'] + inner['count_dagavdeling'])
                    outer['count_dognavdeling'] = (outer['count_dognavdeling'] + inner['count_dognavdeling'])
                    outer['count_osv'] = (outer['count_osv'] + inner['count_osv'])
                    outer['count_outpatient'] = (outer['count_outpatient'] + inner['count_outpatient'])
                    outer['count_inpatient'] = (outer['count_inpatient'] + inner['count_inpatient'])
                    outer['count_lmssciadm'] = (outer['count_lmssciadm'] + inner['count_lmssciadm'])
                    outer['count_inpatient_day'] = (outer['count_inpatient_day'] + inner['count_inpatient_day'])
                    outer['count_inpatient_daynight'] = (outer['count_inpatient_daynight'] + inner['count_inpatient_daynight'])
                    outer['count_terapi'] = (outer['count_terapi'] + inner['count_terapi'])
                    outer['count_undersokelse'] = (outer['count_undersokelse'] + inner['count_undersokelse'])
                    outer['count_radge'] = (outer['count_radge'] + inner['count_radge'])
                    outer['count_behplanlegging'] = (outer['count_behplanlegging'] + inner['count_behplanlegging']) 
                    outer['count_ikkemott'] = (outer['count_ikkemott'] + inner['count_ikkemott'])
                    outer['count_others'] = (outer['count_others'] + inner['count_others'])
                    outer['episode_start_date'] = min(outer['episode_start_date'], inner['episode_start_date'])
                    outer['episode_end_date'] = max(outer['episode_end_date'], inner['episode_end_date']) 
                    j += 1
                    merged = True
                else:
                    break
            if merged:
                merged_episodes.append(outer.to_dict())
            else:
                merged_episodes.append(outer.to_dict())
            i = j

    merged_episodes = pd.DataFrame(merged_episodes)

In [14]:
merged_episodes.loc[merged_episodes['pasient'] == 6586, 
                            ['episode_start_date', 'episode_end_date', 'episode_order', 'tillnextepisode']]

,episode_start_date,episode_end_date,episode_order,tillnextepisode
954,2009-06-30,2010-04-07,1,1090.0
955,2013-04-01,2013-07-07,3,-3.0
956,2013-09-30,2014-04-03,4,270.0
957,2014-12-29,2015-07-02,5,NaN


In [15]:
merged_episodes.drop(columns=['tillnextepisode','starttillnextstart', 'length_of_episode'], inplace=True)
norm_episodes.drop(columns=['tillnextepisode','starttillnextstart', 'length_of_episode'], inplace=True)

<ipython-input-15-09448924660f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  norm_episodes.drop(columns=['tillnextepisode','starttillnextstart', 'length_of_episode'], inplace=True)


In [16]:
merged_episodes.head()

,pasient,episode_id,age,remaining_time_countdown,var_no_dates_permonth,gender,episode_order,islast,closingcode,aftercode,...,count_inpatient,count_lmssciadm,count_inpatient_day,count_inpatient_daynight,count_terapi,count_undersokelse,count_radge,count_behplanlegging,count_ikkemott,count_others
0,19017,23004,0,5,1.000850,F,2,1,9,5,...,0,0,0,0,9,6,0,5,0,1
1,15913,6,11,86,0.911606,M,1,0,1,4,...,0,0,0,0,31,8,3,6,1,1
2,15913,9079,13,17,1.004467,M,3,1,1,5,...,446,0,0,446,940,179,198,147,23,126
3,8327,16823,9,26,0.897503,M,2,1,1,1,...,0,0,0,0,43,31,16,8,3,6
4,16210,2449,8,35,0.779520,M,2,1,2,4,...,0,0,0,0,44,17,12,6,3,4


In [17]:
norm_episodes.head()

,pasient,episode_id,age,remaining_time_countdown,var_no_dates_permonth,gender,episode_order,islast,closingcode,aftercode,...,count_inpatient,count_lmssciadm,count_inpatient_day,count_inpatient_daynight,count_terapi,count_undersokelse,count_radge,count_behplanlegging,count_ikkemott,count_others
1,14049,5,15,2,0.000000,M,1,1,1,4,...,0,0,0,0,9,4,11,0,0,0
3,411,7,12,86,0.489898,M,1,1,1,5,...,0,0,0,0,2,4,23,2,0,2
4,18248,8,16,137,1.603567,F,1,1,1,1,...,0,0,0,0,4,6,0,2,0,0
5,12345,9,16,2,1.806166,F,2,1,4,2,...,48,0,0,48,194,69,64,45,17,14
6,8734,14,7,20,0.640095,M,1,1,1,2,...,0,0,0,0,10,2,0,0,1,1


In [18]:
# an example of normal episodes that was not the case of merge
merged_episodes[merged_episodes['pasient'] == 469]

,pasient,episode_id,age,remaining_time_countdown,var_no_dates_permonth,gender,episode_order,islast,closingcode,aftercode,...,count_inpatient,count_lmssciadm,count_inpatient_day,count_inpatient_daynight,count_terapi,count_undersokelse,count_radge,count_behplanlegging,count_ikkemott,count_others
737,469,28333,16,11,0.767125,F,7,1,1,5,...,198,0,0,198,200,91,79,33,14,36


In [19]:
norm_episodes[norm_episodes['pasient'] == 1]

,pasient,episode_id,age,remaining_time_countdown,var_no_dates_permonth,gender,episode_order,islast,closingcode,aftercode,...,count_inpatient,count_lmssciadm,count_inpatient_day,count_inpatient_daynight,count_terapi,count_undersokelse,count_radge,count_behplanlegging,count_ikkemott,count_others


In [20]:
# calculate tillnextepisode, starttillnextstart and Length_of_Episode for merged_episodes
# Convert date columns to datetime objects
merged_episodes['episode_start_date'] = pd.to_datetime(merged_episodes['episode_start_date'])
merged_episodes['episode_end_date'] = pd.to_datetime(merged_episodes['episode_end_date'])

# Sort the DataFrame by 'pasient' and 'episode_start_date' for accurate calculations

merged_episodes = merged_episodes.sort_values(by=['pasient', 'episode_start_date'])

# Calculate the time till the next episode's start for each patient
merged_episodes['tillnextepisode'] = (merged_episodes.groupby('pasient')['episode_start_date'].shift(-1) - merged_episodes['episode_end_date']).dt.days

# Calculate the time between the current episode's start and the next episode's start for each patient
merged_episodes['starttillnextstart'] = (merged_episodes.groupby('pasient')['episode_start_date'].shift(-1) - merged_episodes['episode_start_date']).dt.days

# Calculate the length of each episode
merged_episodes['Length_of_Episode'] = (merged_episodes['episode_end_date'] - merged_episodes['episode_start_date']).dt.days + 1

In [21]:
merged_episodes[merged_episodes['pasient'] == 6586]

,pasient,episode_id,age,remaining_time_countdown,var_no_dates_permonth,gender,episode_order,islast,closingcode,aftercode,...,count_inpatient_daynight,count_terapi,count_undersokelse,count_radge,count_behplanlegging,count_ikkemott,count_others,tillnextepisode,starttillnextstart,Length_of_Episode
954,6586,20854,12,83,1.615549,F,1,0,1,1,...,0,17,11,9,6,0,1,1090.0,1371.0,282
955,6586,27271,15,44,1.224745,F,3,0,1,2,...,60,25,20,11,1,0,7,85.0,182.0,98
956,6586,12380,16,35,0.755929,F,4,0,1,1,...,0,36,5,3,1,1,1,270.0,455.0,186
957,6586,23951,17,20,1.160577,F,5,1,1,1,...,19,9,5,4,1,0,0,NaN,NaN,186


In [22]:
# calculate tillnextepisode, starttillnextstart and Length_of_Episode for
# Convert date columns to datetime objects
norm_episodes['episode_start_date'] = pd.to_datetime(norm_episodes['episode_start_date'])
norm_episodes['episode_end_date'] = pd.to_datetime(norm_episodes['episode_end_date'])
norm_episodes = norm_episodes.sort_values(by=['pasient', 'episode_start_date'])
# Calculate the time till the next episode's start for each patient
norm_episodes['tillnextepisode'] = (norm_episodes.groupby('pasient')['episode_start_date'].shift(-1) - norm_episodes['episode_end_date']).dt.days

# Calculate the time between the current episode's start and the next episode's start for each patient
norm_episodes['starttillnextstart'] = (norm_episodes.groupby('pasient')['episode_start_date'].shift(-1) - norm_episodes['episode_start_date']).dt.days

#Calculate the length of each episode in the norm_episodes
norm_episodes['Length_of_Episode'] = (norm_episodes['episode_end_date'] - norm_episodes['episode_start_date']).dt.days + 1


<ipython-input-22-74cb3480eb1f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  norm_episodes['episode_start_date'] = pd.to_datetime(norm_episodes['episode_start_date'])
<ipython-input-22-74cb3480eb1f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  norm_episodes['episode_end_date'] = pd.to_datetime(norm_episodes['episode_end_date'])


In [23]:
len(norm_episodes[norm_episodes['Length_of_Episode'] < 0]) #NULL
len(norm_episodes[norm_episodes['Length_of_Episode'] == 0]) # 8 episodes
# some epiosdes like {7841, 18989, 16189, 12388, 21490, 2771,5989, 2482}  are normal epiosdes but start is after end (almost one day)
# to fix this we change the start and end
start_after_end = norm_episodes[norm_episodes['Length_of_Episode'] == 0]
set_id = start_after_end['episode_id']


for episode_id in set_id:
    # Find the corresponding rows in norm_episodes and start_after_end
    norm_row = norm_episodes[norm_episodes['episode_id'] == episode_id]
    sae_row = start_after_end[start_after_end['episode_id'] == episode_id]
    
    # Get the values to replace
    new_start_date = sae_row['episode_end_date'].values[0]
    new_end_date = sae_row['episode_start_date'].values[0]
    
     # Update the values in norm_episodes using .loc[]
    norm_episodes.loc[norm_row.index, 'episode_start_date'] = new_start_date
    norm_episodes.loc[norm_row.index, 'episode_end_date'] = new_end_date


#Re-Calculate the length of each episode in the norm_episodes
norm_episodes['Length_of_Episode'] = (norm_episodes['episode_end_date'] - norm_episodes['episode_start_date']).dt.days + 1

norm_episodes[norm_episodes['Length_of_Episode']==0] # now null


,pasient,episode_id,age,remaining_time_countdown,var_no_dates_permonth,gender,episode_order,islast,closingcode,aftercode,...,count_inpatient_daynight,count_terapi,count_undersokelse,count_radge,count_behplanlegging,count_ikkemott,count_others,tillnextepisode,starttillnextstart,Length_of_Episode


In [24]:
# count of cases_of_merge: 1378
len(merged_episodes) #  after merge, it became 752
#len(norm_episodes) # 23453 episodes

1294

In [25]:
# Merge to epiosdes data frames
# axis=0 along rows
full_episodes = pd.concat([norm_episodes, merged_episodes], axis=0, join='outer', ignore_index=False)

In [26]:
full_episodes.drop_duplicates(inplace=True)

In [27]:
len(full_episodes) 

17433

In [28]:
full_episodes[full_episodes['tillnextepisode']<0].head()

,pasient,episode_id,age,remaining_time_countdown,var_no_dates_permonth,gender,episode_order,islast,closingcode,aftercode,...,count_inpatient_daynight,count_terapi,count_undersokelse,count_radge,count_behplanlegging,count_ikkemott,count_others,tillnextepisode,starttillnextstart,Length_of_Episode


In [29]:
#looking at 'Length_of_Episode'
full_episodes[(full_episodes['Length_of_Episode'] == 0)] # NULL
full_episodes[(full_episodes['Length_of_Episode'] < 0)] # NULL
full_episodes["Length_of_Episode"].describe()

count    17433.000000
mean       807.901279
std        934.755295
min          1.000000
25%        187.000000
50%        468.000000
75%       1010.000000
max       6935.000000
Name: Length_of_Episode, dtype: float64

# Store the result of merging in dataframe and CSV: full_episodes

In [31]:
full_episodes.to_csv("/home/kabank/workbench/.conda/analysis/Resources/full_episodes.csv")